In [ ]:
import math
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import model_from_json


In [ ]:
DATASET_PATH = "../input/392-crypto-currency-pairs-at-minute-resolution/btcusd.csv"

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
df=pd.read_csv(DATASET_PATH)
data = df['close']

In [ ]:
data = data.tolist()

In [ ]:
len(data)

In [ ]:
data = data[-1500000:]

In [ ]:
len(data)

In [ ]:
data = np.array(data)

In [ ]:
print(data)

In [ ]:
print(np.min(data), np.max(data))

In [ ]:
def make_data(data,train):
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    
    X_total = []
    y_total = []
    
    X_eql = []
    X_inc = []
    X_dec = []

    y_eql = []
    y_inc = []
    y_dec = []
    
    len_eql = 0
    len_inc = 0
    len_dec = 0
    
    for i in range(len(data)-401):
        
        X = data[i:i+400]
        mx = np.max(X)
        X = (X/mx)*0.9
        
        
        if data[i+400]>data[i+399]:
            y = [1,0,0]
            y_inc.append(y)
            len_inc+=1
            X_inc.append(X)
        if data[i+400]==data[i+399]:
            y = [0,1,0]
            y_eql.append(y)
            len_eql+=1
            X_eql.append(X)
        if data[i+400]<data[i+399]:
            y = [0,0,1]
            y_dec.append(y)
            len_dec+=1
            X_dec.append(X)
        
    length = min(len_inc,len_eql,len_dec)

    y_total = y_inc[:length] + y_eql[:length] + y_dec[:length]
    X_total = X_inc[:length] + X_eql[:length] + X_dec[:length]

    len_train = int(len(y_total)*train)
        
    X_train = X_total[:len_train]
    y_train = y_total[:len_train]
    
    X_val = X_total[-len_train:]
    y_val = y_total[-len_train:]
    
    X_train = np.array(X_train).reshape(-1,400,1)
    y_train = np.array(y_train).reshape(-1,3)
    X_val = np.array(X_val).reshape(-1,400,1)
    y_val = np.array(y_val).reshape(-1,3)


    
    return X_train, y_train, X_val, y_val


In [ ]:
X_train, y_train, X_val, y_val = make_data(data,0.96)

In [ ]:
y_train[0]

In [ ]:
X_train[0]

In [ ]:
print(type(y_train[0]))

In [ ]:
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
y_train[10:20]

In [ ]:
model = Sequential()
model.add(LSTM(128,
          input_shape=(400, 1)))
model.add(Dense(100,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# early_stop = EarlyStopping(monitor='val_loss', patience=20)

checkpoint = ModelCheckpoint("model_best.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=30, 
                    batch_size=50,
                    validation_data=(X_val,y_val),
                    callbacks=[checkpoint])

In [ ]:
model_json = model.to_json()
with open("model_final.json", "w") as json_file : 
    json_file.write(model_json)

In [ ]:
model.save_weights("model_final.h5")

In [ ]:
# with open("./model_final.json", "r") as fp:
#   model_test = model_from_json(fp.read())

# # Load weights to new model
# model_test.load_weights("./model_final.h5")

In [ ]:
# df=pd.read_csv(DATASET_PATH)
# data_test = df['close']

In [ ]:
# data_test = data_test.tolist()

In [ ]:
# data_test = data_test[800000:1300000]

In [ ]:
# data_test = np.array(data_test)

In [ ]:
# plt.plot(range(len(data_test)),data_test)
# # plt.plot(range(len(train_data)),train_data)
# plt.show()

In [ ]:
# X_train_test, y_train_test, X_val_test, y_val_test = make_data(data_test,0.95)

In [ ]:
# all = 0
# counter = 0
# test = 0
# for i,y_guess in enumerate(model_test.predict(X_train_test)):

#     if y_train_test[i][0]>X_train_test[i][-1][0]:
#         all+=1
#         if y_guess[0]>X_train_test[i][-1][0]:
#             counter+=1
    
#     if y_train_test[i][0]==X_train_test[i][-1][0]:
#         test+=1

# print(counter/all)
# print(test/all)

In [ ]:
# all = 0
# counter = 0
# for i,y_guess in enumerate(model_test.predict(X_train_test)):
    
#     if not y_train_test[i][0]==X_train_test[i][-1][0]:
#         all+=1

#     if y_train_test[i][0]<X_train_test[i][-1][0]:
#         if y_guess[0]<X_train_test[i][-1][0]:
#             counter+=1
#     if y_train_test[i][0]>X_train_test[i][-1][0]:
#         if y_guess[0]>X_train_test[i][-1][0]:
#             counter+=1

# print(counter/all)

In [ ]:
# predict = model_test.predict(X_train_test)

In [ ]:
# a=1
# for i,y_guess in enumerate(predict):
#     if y_guess[0]<X_train_test[i][-1][0]:
#         a *= (y_train_test[i][0]/X_train_test[i][-1][0])

In [ ]:
# print(a)